In [78]:
import pandas as pd
import json
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [314]:
with open('scrapped_adds.json') as json_file:
    data = json.load(json_file)
    
data = pd.DataFrame.from_dict(data, orient='index')

#ids
data['id'] = [i for i in range(0,len(data))]
data['url']= [data['url'][i][0] for i in range(0, len(data))]

#operación venta
data.loc[data['operacion'] == 'Desde',['operacion']] = 'Venta'

#Direccion, colonias y alcaldias
data['direccion'] = [data['direccion'][i].replace('\n ', '').replace('\n', '') for i in range(0, len(data))]
data['alcaldia'] = [data['direccion'][i].split(',')[-1] for i in range(0, len(data))]
data['colonia'] = 'N/A'
data['moneda'] = 'N/A'
data['precio_MXN'] = 'N/A'

for i in range(0,len(data)):
    
    #colonia
    try:
        data['colonia'][i] = data['direccion'][i].split(', ')[-2]
    except:
        print('Error in:', i, ': ', data['direccion'][i])
    
    #antiguedad
    try:
        if len(data['antiguedad'][i]) == 0:
            data['antiguedad'][i] = 'N/A'
        else:
            data['antiguedad'][i] = data['antiguedad'][i].replace('Antigüedad', '')
            if (data['antiguedad'][i] == 'A estrenar') | (data['antiguedad'][i] == 'En construcción'):
                data['antiguedad'][i] = '0'
            if len(data['antiguedad'][i]) == 4:
                data['antiguedad'][i] = 2021 - int(data['antiguedad'][i])     
    except:
        print(i, 'not able to get antiguedad')
    
    #superficie total
    try:
        if len(data['superficie_total'][i]) == 0:
            data['superficie_total'][i] = 'N/A'
        else:
            data['superficie_total'][i] = data['superficie_total'][i].replace('m² Total', '')         
    except:
        print(i, 'not able to get superficie')
    
    #superficie cubierta
    try:
        if len(data['superficie_cubierta'][i]) == 0:
            data['superficie_cubierta'][i] = 'N/A'
        else:
            data['superficie_cubierta'][i] = data['superficie_cubierta'][i].replace('m² Construido', '')     
    except:
        print(i, 'not able to get superficie cubierta')
        
    #Baños
    try:
        if len(data['baños'][i]) == 0:
            data['baños'][i] = 'N/A'
        else:
            data['baños'][i] = data['baños'][i].replace('Baños', '').replace('Baño', '')    
    except:
          print(i, 'not able to get baños')
    
    #dormitorios
    try:
        if len(data['dormitorios'][i]) == 0:
            data['dormitorios'][i] = 'N/A'
        else:
            data['dormitorios'][i] = data['dormitorios'][i].replace('Recámaras', '').replace('Recámara', '')    
    except:
          print(i, 'not able to get dormitorios')
            
    #estacionamientos
    try:
        if len(data['cocheras'][i]) == 0:
            data['cocheras'][i] = 'N/A'
        else:
            data['cocheras'][i] = data['cocheras'][i].replace('Estacionamientos', '').replace('Estacionamiento', '')    
    except:
          print(i, 'not able to get estacionamientos')
            
    #mantenimiento
    try:
        if len(data['mantenimiento'][i]) == 0:
            data['mantenimiento'][i] = 'N/A'
        else:
            data['mantenimiento'][i] = data['mantenimiento'][i].replace('MN ', '').replace(',', '')    
    except:
          print(i, 'not able to get estacionamientos')
     
    #precios
    try:
        if len(data['precio'][i]) == 0:
            data['precio'][i] = 'N/A'
        else:
            if data['precio'].str.contains('MN ')[i]:
                data['moneda'][i] = 'MXN'
            else: 
                if data['precio'].str.contains('USD ')[i]:
                    data['moneda'][i] = 'USD'
                
        data['precio'][i] = data['precio'][i].replace('MN ', '').replace(',', '')
        data['precio'][i] = data['precio'][i].replace('USD ', '').replace(',', '') 
    except:
          print(i, 'not able to get precios')
    
    #lat lng
    try:
        if len(data['anunciante'][i]) == 0:
            data['anunciante'][i] = 'N/A'
    except:
        print(i, 'not able to get anunciante')
    
    #lat lng
    try: 
        if len(data['lat'][i]) == 0:
            data['lat'][i] = data['google_src'][i].split('center=')[1].split('&')[0].split(',')[0]
        if len(data['lng'][i]) == 0:
            data['lng'][i] = data['google_src'][i].split('center=')[1].split('&')[0].split(',')[1]
    except:
        print(i, 'not able to get lat/lng')
        

data['antiguedad']= pd.to_numeric(data['antiguedad'], errors= 'coerce')
data['superficie_total'] = pd.to_numeric(data['superficie_total'], errors= 'coerce') 
data['superficie_cubierta'] = pd.to_numeric(data['superficie_cubierta'], errors= 'coerce')
data['baños'] = pd.to_numeric(data['baños'], errors= 'coerce')
data['dormitorios'] = pd.to_numeric(data['dormitorios'], errors= 'coerce')
data['cocheras'] = pd.to_numeric(data['cocheras'], errors= 'coerce')
data['precio'] = pd.to_numeric(data['precio'], errors= 'coerce')
data['mantenimiento'] = pd.to_numeric(data['mantenimiento'], errors= 'coerce')
data['lat'] = pd.to_numeric(data['lat'], errors= 'coerce')
data['lng'] = pd.to_numeric(data['lng'], errors= 'coerce')

data['precio_m2'] = data['precio'] / data['superficie_total']
        
data = data[['id','precio_m2', 'precio','moneda', 'precio_MXN', 'mantenimiento', 'superficie_total', 'superficie_cubierta',
             'colonia', 'alcaldia', 'direccion',
            'dormitorios', 'baños', 'cocheras','anunciante', 'url', 'google_src', 'lat', 'lng']]

for i in range(0, len(data)):
    if data['moneda'][i] == 'USD':
        data.iloc[i,3] = data['precio'][i] * 19.8
    else:
        if data['moneda'][i] == 'MXN':
            data.iloc[i,3] = data['precio'][i]
        else:
            data.iloc[i,3] = np.nan



data.head()

Error in: 857 :  Dirección oculta
857 not able to get lat/lng
Error in: 3875 :  Dirección oculta
3875 not able to get lat/lng


,id,precio_m2,precio,moneda,precio_MXN,mantenimiento,superficie_total,superficie_cubierta,colonia,alcaldia,direccion,dormitorios,baños,cocheras,anunciante,url,google_src,lat,lng
0,0,NaN,4100000,4100000,N/A,NaN,NaN,NaN,San Ángel,Alvaro Obregón,"Boulevard Adolfo Lopez Mateos 1970 Los Alpes, ...",NaN,NaN,NaN,N/A,https://www.inmuebles24.com/propiedades/nova-s...,//maps.google.com/maps/api/staticmap?center=19...,19.359719,-99.192007
1,1,NaN,7900000,7900000,N/A,NaN,NaN,NaN,Bosques de las Lomas,Cuajimalpa de Morelos,"Bosque de las Lomas, Bosques de las Lomas, Cua...",NaN,NaN,NaN,N/A,https://www.inmuebles24.com/propiedades/nova-l...,//maps.google.com/maps/api/staticmap?center=19...,19.390416,-99.255000
2,2,NaN,18407502,18407502,N/A,NaN,NaN,NaN,Polanco,Miguel Hidalgo,"Alfredo de Musset 326, Polanco, Miguel Hidalgo",NaN,NaN,NaN,Musset 326,https://www.inmuebles24.com/propiedades/musset...,//maps.google.com/maps/api/staticmap?center=19...,19.436881,-99.196634
3,3,NaN,4676520,4676520,N/A,NaN,NaN,NaN,Roma Norte,Cuauhtémoc,"Coahuila 193, Roma Norte, Cuauhtémoc",NaN,NaN,NaN,N/A,https://www.inmuebles24.com/propiedades/pre-ve...,//maps.google.com/maps/api/staticmap?center=19...,19.410923,-99.165172
4,4,NaN,3317050,3317050,N/A,NaN,NaN,NaN,San Pedro de los Pinos,Benito Juárez,"PATRIOTISMO 307, San Pedro de los Pinos, Benit...",NaN,NaN,NaN,N/A,https://www.inmuebles24.com/propiedades/gran-p...,//maps.google.com/maps/api/staticmap?center=19...,19.393290,-99.181658


In [313]:
data.to_csv('data_processed.csv', sep='|')